In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

# Load dataset

In [10]:
words = pd.read_csv("./datasets/words.txt", header=None)
words.columns = ["words"]
words

,words
0,rossa
1,jetty
2,wizzo
3,cuppa
4,cohoe
...,...
14850,dunny
14851,decal
14852,fungs
14853,cadgy


In [11]:
for i in range(5):
    words[f"letter_{i+1}"] = words["words"].apply(lambda x: x[i])

words

,words,letter_1,letter_2,letter_3,letter_4,letter_5
0,rossa,r,o,s,s,a
1,jetty,j,e,t,t,y
2,wizzo,w,i,z,z,o
3,cuppa,c,u,p,p,a
4,cohoe,c,o,h,o,e
...,...,...,...,...,...,...
14850,dunny,d,u,n,n,y
14851,decal,d,e,c,a,l
14852,fungs,f,u,n,g,s
14853,cadgy,c,a,d,g,y


In [13]:
letter_cols = [f"letter_{i+1}" for i in range(5)]

['letter_1', 'letter_2', 'letter_3', 'letter_4', 'letter_5']

In [43]:
all_letters = pd.concat([pd.DataFrame({"letters": words[col], "position": col, "count": 1}) for col in letter_cols])
letter_counts = all_letters.groupby(["letters", "position"]).sum()
letter_counts.reset_index(inplace=True)
total_counts = letter_counts.groupby("letters")[["count"]].sum()
total_counts.columns = ["total_counts"]
total_counts.reset_index(inplace=True)
letter_counts = letter_counts.merge(total_counts, on="letters")
letter_counts.sort_values(["total_counts", "position"], ascending=[False, True], inplace=True)
letter_counts

,letters,position,count,total_counts
20,e,letter_1,330,7455
21,e,letter_2,1857,7455
22,e,letter_3,998,7455
23,e,letter_4,2531,7455
24,e,letter_5,1739,7455
...,...,...,...,...
80,q,letter_1,103,145
81,q,letter_2,16,145
82,q,letter_3,18,145
83,q,letter_4,3,145


In [44]:
px.bar(letter_counts, x="letters", y="count", color="position", title="Occurance counts of letters")

In [45]:
vowel_list = ["a", "e", "i", "o", "u", "y"]
vowel_counts = letter_counts[letter_counts["letters"].isin(vowel_list)]
px.bar(vowel_counts, x="letters", y="count", color="position", title="Occurance counts of vowels")

In [46]:
vowel_list = ["a", "e", "i", "o", "u", "y"]
vowel_counts = letter_counts[~letter_counts["letters"].isin(vowel_list)]
px.bar(vowel_counts, x="letters", y="count", color="position", title="Occurance counts of vowels")